In [39]:
import pandas as pd

In [40]:
df = pd.read_csv('kb_metadata_dan_filtered_openrefine_v_1.csv')

In [41]:
subset = df[['year_st', 'place_st']]

In [42]:
# Remove rows with any NaN values
subset = subset.dropna()

In [43]:
# Notice that there are data errors in the 'year_st' column, so we remove years before 1600 and after 1900.

subset = subset.query('year_st >=1600 and year_st <=1900 ').reset_index(drop=True)

In [44]:
# Sort the dataset based on the 'year_st' column so that lowest value is at the top
subset = subset.sort_values(by='year_st').reset_index(drop=True)

In [45]:
# Extract the 'place' column and convert it to a set
unique_places = list(set(df['place_st']))

In [36]:
#! pip install geopy

In [56]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

# Extract the 'place' column and convert it to a set
unique_places = list(set(subset['place_st']))

# Extract the 'place' column and convert it to a set, removing any NaN or non-string values
unique_places = list(set(subset['place_st'].dropna()))
unique_places = [place for place in unique_places if isinstance(place, str)]

# Initialize the Nominatim geocoder
geolocator = Nominatim(user_agent="your_app_name")

def geocode_place(place_name):
    try:
        location = geolocator.geocode(place_name)
        if location:
            return (location.latitude, location.longitude)
        else:
            return (None, None)
    except GeocoderTimedOut:
        return (None, None)

# Initialize an empty list to store the results
geocoded_places = []
for place_name in unique_places:
    lat, lon = geocode_place(place_name)
    geocoded_places.append([place_name, lat, lon])

# Convert the list to a pandas DataFrame
df_geocoded = pd.DataFrame(geocoded_places, columns=['place_st', 'latitude', 'longitude'])

print(df_geocoded)

             place_st   latitude   longitude
0           Wiesbaden  50.082038    8.241656
1    Nykøbing Jylland        NaN         NaN
2           Stralsund  54.309631   13.082085
3               Tokio  35.682194  139.762221
4         Blair Nebr.  41.544438  -96.125015
..                ...        ...         ...
408   Sevel-Hedegaard        NaN         NaN
409     Frederiksstad  54.376000    9.089772
410         Skovlunde  55.723064   12.402713
411           Herning  56.137976    8.974662
412         Bruxelles  50.855002    4.351233

[413 rows x 3 columns]


In [64]:
# Make a copy before continuering ( not really important ) 
df_geo = df_geocoded.copy()

# Remove rows with any NaN values - not all string values have been recognized and geocoded 
df_geo = df_geo.dropna()

# Perform the join of the subset data frame and the georeferenced dataframe on the 'place_name' column
subset_geocoded = pd.merge(subset, df_geo, on='place_st', how='inner')

# Storing it as a csv file as well
output_file = 'geocoded_places.csv'
subset_geocoded.to_csv(output_file, index=False)
